In [1]:
import sys
sys.path.insert(0, "..")

import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import Model

from dynamic_tcn import DynamicTCN
from classifier import dynamic_tcn_classifier

In [7]:
dtcn = DynamicTCN(use_dynamic_conv=True)

In [3]:
X = np.random.rand(100, 200, 3)
y = (np.random.rand(100) > 0.5).astype(np.uint8)

In [8]:
clf = dynamic_tcn_classifier(input_shape=(None, 3), batch_size=4)

In [9]:
clf.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss="mse")

In [ ]:
clf.fit(X, y, batch_size=4, epochs=1)

 5/25 [=====>........................] - ETA: 1s - loss: 0.5258